In [89]:
from working_with_files import *
from preprocessing import *
import pandas as pd
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [2]:
def sort_by_official_dates(df):
    df1 = pd.DataFrame() # < 2008
    df2 = pd.DataFrame() # 2008 - 2013
    df3 = pd.DataFrame() # 2013 - 2016
    df4 = pd.DataFrame() # > 2016
    date1 = datetime.strptime('15-07-2008', '%d-%m-%Y').date()
    date2 = datetime.strptime('20-02-2013', '%d-%m-%Y').date()
    date3 = datetime.strptime('30-11-2016', '%d-%m-%Y').date()
    for index, row in df.iterrows():
        if int(row['Unnamed: 0']) not in [10995, 11494]:
            try:
                date = datetime.strptime(row['date'].strip(), '%d-%m-%Y').date()
            except ValueError:
                date = datetime.strptime(row['date'], '%m-%Y').date()
            if date <= date1:
                df1 = df1.append(row, ignore_index=True)
            if date > date1 and date <= date2:
                df2 = df2.append(row, ignore_index=True)
            if date > date2 and date <= date3:
                df3 = df3.append(row, ignore_index=True)
            if date > date3:
                df4 = df4.append(row, ignore_index=True)
    return df1, df2, df3, df4

def show_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print('Topic {}:'.format(topic_idx))
        print(' '.join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
    print()

def get_topics(documents, n_topics, n_top_words, models):
    
    random_seed = 42
    
    if 'nmf' in models:
        tfidf = tfidf_vectorizer.fit_transform(documents)
        tfidf_feature_names = tfidf_vectorizer.get_feature_names()
        
        nmf = NMF(n_components=n_topics, random_state=random_seed, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)
        
        print('NMF')
        show_topics(nmf, tfidf_feature_names, n_top_words)
        
    if 'lda' in models:
        tf = tf_vectorizer.fit_transform(documents)
        tf_feature_names = tf_vectorizer.get_feature_names()
        
        lda = LatentDirichletAllocation(n_components=n_topics, max_iter=5, learning_method='online', learning_offset=50., 
                                random_state=random_seed).fit(tf)
        
        print('LDA')
        show_topics(lda, tf_feature_names, n_top_words)

In [3]:
# read df and split it into 4 separate df4 based on publication date
df = pd.read_csv('mfa_texts_df.csv', sep = '\t', encoding = 'utf-8')
sorted_dfs = sort_by_official_dates(df)

In [4]:
# statistics 
n = 0
for df in sorted_dfs:
    print('Corpus {}:'.format(n), len(df), 'texts', len(get_all_texts(df).split()), 'words')
    n += 1

Corpus 0: 1583 texts 845988 words
Corpus 1: 3168 texts 1111520 words
Corpus 2: 3344 texts 1082001 words
Corpus 3: 4277 texts 1420393 words


In [90]:
# get unnecessary symbols to remove them from texts
symbols_to_del = generate_symbols_to_del(get_all_texts(df))
symbols_to_del.extend(list('abcdefghijklmnopqrstuvwxyz'))
print(symbols_to_del)

['4', '0', '*', '9', '2', '#', '_', '3', '@', '6', '\xad', '”', '−', '‑', '’', '№', '.', '<', '/', '„', '!', "'", ':', '1', '\n', '%', '+', ')', '"', '´', '́', '?', '–', '—', '=', '！', '«', '7', '5', '&', '(', '…', '»', ',', ';', '“', '>', '8', '‒', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [5]:
# clean texts from unnecessary symbols

for df in sorted_dfs:
    df['text_clean'] = df['text_lemmatized'].apply(clean, args=(symbols_to_del,))

In [119]:
# get and preprocess stopwords
# source: ruscorpora.ru

stopwords_raw = get_lexicon_from_file('russian_stopwords.txt')
stopwords = []
stopwords_string = ''.join(stopwords_raw)
strings = stopwords_string.split('\t')
for string in strings:
    if string.isalpha():
        stopwords.append(string)

# lemmatize stopwords

stopwords_lemmatized = [lemmatize_pymorphy(word) for word in stopwords]
stopwords_lemmatized = list(set(stopwords_lemmatized))
stopwords = [stopword for stopword in stopwords_lemmatized if stopword.isalpha()]

# stopwords.extend(['наш', 'твой', 'мой', 'свой', 'ваш', 'его', 'их', 'ее', 'над', 'чей', 'между', 'год', 'вопрос', 'страна', 
 #                'россия', 'российский', 'такой', 'государство', 'другой', 'также', 'рф', 'мид', 'сегодня', 'два', 'катерина',
 #                'спецкор', 'дать', 'прокомментировать', 'интервью', 'федерация', 'лавров', 'сторона', 'российско', 'господин',
 #                'рассказывать', 'министр', 'встреча', 'состояться', 'сайт', 'итог', 'официальный', 'глава', 'иностранный',
 #                'дело', 'москва', 'александр', 'курировать', 'иванов', 'делегации', 'заместитель', 'комментировать', 'уважаемый',
 #                'президент', 'москве', 'государственный' 'лавр', 'москвы', 'принять', 'участие', 'мероприятие', 'россии', 
 #                 'медведев', 'пресс', 'посетить'])

stopwords.extend('''б г д е ж з й л м н п р т ф х ц ч ш щ ъ ы ь э ю
                наш твой мой свой ваш его их ее над чей такой другой между год страна государство также все отношение г-н 
                россия российский сторона де два дж мид внешнеполитический интервью министр газета иностранный дело
                катерина спецкор новость рассказывать подобный кроме кстати пресс-конференция москва сайт лавров
                медведев путин сайт подробный рф s форум сердечно добрый желать приветствовать сам ясный кто-то круглый
                ясность ясно ярмарка ярко яркий говорить хотеть официальный еще какой-то об'''.split())

print(stopwords)

['с', 'вот', 'бы', 'сказать', 'есть', 'по', 'весь', 'это', 'не', 'из', 'очень', 'тот', 'у', 'ли', 'ещё', 'надо', 'а', 'когда', 'уже', 'кто', 'я', 'себя', 'человек', 'же', 'ни', 'мочь', 'более', 'без', 'от', 'после', 'или', 'чтобы', 'но', 'за', 'ничто', 'этот', 'к', 'то', 'тем', 'потому', 'он', 'в', 'вы', 'один', 'потом', 'до', 'теперь', 'для', 'можно', 'только', 'как', 'тогда', 'там', 'чем', 'она', 'они', 'даже', 'тут', 'ты', 'где', 'который', 'так', 'здесь', 'на', 'быть', 'ну', 'через', 'если', 'о', 'при', 'мы', 'что', 'и', 'нет', 'под', 'раз', 'время', 'б', 'г', 'д', 'е', 'ж', 'з', 'й', 'л', 'м', 'н', 'п', 'р', 'т', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'наш', 'твой', 'мой', 'свой', 'ваш', 'его', 'их', 'ее', 'над', 'чей', 'такой', 'другой', 'между', 'год', 'страна', 'государство', 'также', 'все', 'отношение', 'г-н', 'россия', 'российский', 'сторона', 'де', 'два', 'дж', 'мид', 'внешнеполитический', 'интервью', 'министр', 'газета', 'иностранный', 'дело', 'катерина', 'с

In [120]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=10, stop_words=stopwords, token_pattern=r"\b[\w-]+\b")
tf_vectorizer = CountVectorizer(max_df=0.9, min_df=10, stop_words=stopwords, token_pattern=r"\b[\w-]+\b")

In [123]:
get_topics(sorted_dfs[0].loc[:, 'text_clean'], 36, 15, ['nmf']) # до 2008

NMF
Topic 0:
международный развитие политика новый мир мировой интерес решение проблема становиться работа экономический сотрудничество безопасность важный
Topic 1:
израиль палестинец дорожный карта израильский палестино-израильский палестинский четверка посредник квартет урегулирование пна израильтянин ближневосточный аббас
Topic 2:
сотрудничество вопрос обсуждать переговоры визит президент договариваться двусторонний встреча договоренность сегодня взаимодействие торговый-экономический сфера коллега
Topic 3:
палестинский пна аббас хамас администрация территория межпалестинский глава правительство национальный народ израиль обстановка палестинец мирный
Topic 4:
европа совет се европейский председательство конвенция комитет парламентский право правовой общеевропейский ассамблея разделительный организация демократия
Topic 5:
исламский цивилизация оик религия ислам религиозный православный мусульманский мир церковь культура мусульманин межцивилизационный конференция диалог
Topic 6:
соотеч

In [102]:
get_topics(sorted_dfs[1].loc[:, 'text_clean'], 39, 15, ['nmf']) # 2008 - 2013

NMF
Topic 0:
международный мир развитие новый глобальный мировой становиться безопасность работа интерес основа политика решение система важный
Topic 1:
осетия южный абхазия республика закавказье женевский граница грузия абхазский неприменение дискуссия югоосетинский сила соглашение цхинвал
Topic 2:
сотрудничество двусторонний взаимодействие обсуждать переговоры вопрос сфера область торговый-экономический коллега связь развитие ход визит совместный
Topic 3:
соотечественник русский рубеж язык конференция зарубежный конгресс проживать всемирный правительственный диаспора община координационный комиссия работа
Topic 4:
гражданин посольство консульский ребенок власть суд информация сотрудник ситуация находиться россиянин необходимый виза сми семья
Topic 5:
ядерный договор дняо разоружение нераспространение оружие конференция вооружение снв стратегический сокращение безопасность наступательный двзяи атомный
Topic 6:
снг содружество заседание глава совет государство-участник казахстан рассма

In [104]:
get_topics(sorted_dfs[2].loc[:, 'text_clean'], 36, 15, ['nmf']) # 2013 - 2016

NMF
Topic 0:
международный развитие сотрудничество регион интерес работа экономический сегодня новый усилие безопасность мир основа проблема важный
Topic 1:
сирия сирийский оппозиция боевик гуманитарный алеппо сара террорист дамаск вооруженный район группировка джабхат правительство группа
Topic 2:
украина украинский киев власть юго-восток киевский гуманитарный минский ситуация население донбасс заявление донецкий донецк действие
Topic 3:
визит сотрудничество двусторонний федерация переговоры вопрос встреча ход обсуждать актуальный состояться взаимодействие повестка республика торговый-экономический
Topic 4:
погибший выздоровление соболезнование пострадавший осуждать решительно террористический погибать выражать близкие скорый родные теракт ранение результат
Topic 5:
сша американский вашингтон администрация санкция против госдепартамент российский-американский керри про заявление обама белый санкционный госсекретарь
Topic 6:
оон резолюция сб совет безопасность генеральный право междуна

In [124]:
get_topics(sorted_dfs[3].loc[:, 'text_clean'], 35, 15, ['nmf']) # С 2016

NMF
Topic 0:
международный интерес регион безопасность сотрудничество развитие работа мир проблема экономический число важный решение новый основа
Topic 1:
сша американский вашингтон американец санкция договор госдепартамент администрация военный против ракета дрсмд заявление вооружение обвинение
Topic 2:
сирийский сирия турция астанинский процесс урегулирование оон иран резолюция конгресс сб сочи оппозиция астана сириец
Topic 3:
украина украинский киев киевский донбасс власть минский крым закон режим порошенко миссия действие нынешний вышинский
Topic 4:
визит двусторонний сотрудничество переговоры федерация республика актуальный региональный обсуждать вопрос рабочий повестка состояться ход международный
Topic 5:
мероприятие участие молодой дипломат федерация представитель дипломатический принимать состояться федеральный министерство посвящать приглашать совет посол
Topic 6:
сирия сирийский террорист боевик провинция гуманитарный район зона житель лагерь пункт военный помощь алеппо дам

In [46]:
n = 0
for text in sorted_dfs[2]['text_lemmatized']:
    if 'ливан' in text and 'единство' in text:
        print(text, '\n')
        n += 1
n

31 октябрь в соответствие с процедура, предусмотренный конституция ливанский республика, по итог голосование в парламент страна состояться избрание президент ливан. они становиться известный христианский политик, основатель ливанский свободный патриотический движение генерал мишель аун. голос в его поддержка отдавать 83 депутат 128-местный ливанский парламент. м.аун приносить присяга и вступать должность.
с удовлетворение отмечать преодоление длиться с май 2014 г. президентский кризис в ливан. рассчитывать, что за это последовать востребовать весь гражданин страна шаг по укрепление устойчивый стабильность и обеспечение устойчивый развитие. в москва неизменно исходить из то, что сохранение и эффективный работа ливанский государственный институт являться залог внутренний мир и единство этот многоконфессиональный страна. в контакт со весь без исключение ливанский общественно-политический сила российский сторона активно содействовать достижение национальный консенсус в ливан.
россия намере

16